<a href="https://colab.research.google.com/github/keiriina/mangroves-chatbot/blob/main/chatbot_with_cohere_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mangrove Chatbot

> Hi! This is a chatbot that answers basic questions about mangroves.


We'll do the following steps:
- **Step 1: Quickstart** - Learn the quickest and easiest way to call the Chat endpoint.
- **Step 2: Creating a custom system prompt** - Steer a chatbot's response toward certain styles, personas, or other characteristics.
- **Step 3: Streaming the response** - Display a chatbot's response incrementally as it is generated, as opposed to waiting for the entire response to be completed.
- **Step 4: Building the Chat History** - Explore how to get the chatbot to maintain the context of the conversation.
- **Step 5: Retrieval-Augmented Generation (RAG): Adding a knowledge base** - Contextualize responses by adding a source of truth where the chatbot should find answers

In [9]:
# install
! pip install cohere python-dotenv requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.3/295.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 53.5 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.1
    Uninstalling httpx-sse-0.4.1:
      Successfully uninstalled httpx-sse-0.4.1


In [10]:
# import
import cohere
import os
import dotenv
import requests

In [12]:
# load env
dotenv.load_dotenv()

co = cohere.ClientV2(os.getenv("COHERE_API_KEY"))

## Define the System Prompt

To get started with Chat, we add a user message to the `messages` parameter and define the model type in the `model` parameter. We then call the Chat endpoint through the client we created earlier.

The response contains several objects. For simplicity, what we want right now is the `text` object.

Here's an example of the assistant responding to a new hire's query asking for help to make introductions.

In [6]:
# Add the user message
message = "I'm teaching about Mangroves in the Philippines. Can you tell me more about mangroves?"

# Create a custom system message
system_message="""## Task and Context
You are a speaker for the locals of a community that needs to conserve their mangrove systems.

## Style Guide
Speak in a way that can be easily understood by anyone, even people who are not expert in English. Be precise."""

# Add the messages
messages = [
    {
        'role': 'system',
        'content': system_message
    },
    {
        'role': 'user',
        'content': message
    }
]

# Generate the response by streaming it
response = co.chat_stream(model="command-r-plus-08-2024",
                          messages=[
                              {
                                  'role':'user',
                                  'content': message
                              }
                          ])

for event in response:
    if event.type == "content-delta":
        print(event.delta.message.content.text, end="")

Mangroves are unique and highly specialized coastal ecosystems found in tropical and subtropical regions. Here's some information about mangroves, specifically focusing on the Philippines:

**Definition and Characteristics:**
- Mangroves refer to both the habitat and the diverse group of trees and shrubs that thrive in saline coastal habitats. They are characterized by their ability to grow in intertidal zones, where they are regularly inundated with seawater.

**Distribution in the Philippines:**
- The Philippines is home to a significant portion of the world's mangrove forests, with an estimated 400,000 hectares of mangroves.
- Mangroves are widely distributed across the Philippine archipelago, particularly in coastal areas and river estuaries. Some of the regions with notable mangrove ecosystems include Palawan, Zamboanga, Bohol, Leyte, Samar, and the Mindanao region.

**Ecological Importance:**
- Mangroves play a critical role in the Philippine ecosystem:
   - **Coastal Protection:

## Build Knowledge Base

In [3]:
! pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 76.0 MB/s eta 0:00:00


In [5]:
import pymupdf

def load_pdf(file_path):
    doc = pymupdf.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_text = load_pdf("mangroves.pdf")


In [14]:
documents = [
    {
        "id": "mangrove-doc",
        "data": {
            "title": "Mangroves in the Philippines",
            "text": pdf_text
        }
    }
]


## Build Conversation Memory

In [16]:
# custom system msg
system_message = """## Task and Context
You are a speaker for the locals of a community that needs to conserve their mangrove systems.

## Style Guide
Speak in a way that can be easily understood by anyone, even people who are not expert in English. Be precise.
"""

# initialize conversation
messages = [
    {
        'role': 'system',
        'content': system_message
    }
]

while True:
    # check if limit true
    user_count = sum(1 for m in messages if m['role'] == 'user')
    if user_count >= 10:
        print("Conversation limit reached (10 exchanges). Please start a new chat.")
        break

    # user input
    user_message = input("You: ")
    if user_message.lower() in ["q"]:
        break

    # append message to convo history
    messages.append({'role': 'user', 'content': user_message})

    # chat stream
    bot_reply = ""
    response = co.chat_stream(
        model="command-r-plus-08-2024",
        messages=messages,
        documents=documents
    )

    for event in response:
        if event.type == "content-delta":
            chunk = event.delta.message.content.text
            print(chunk, end="")
            bot_reply += chunk

    print()

    # bot's reply add to convo history
    messages.append({'role': 'assistant', 'content': bot_reply})


You: q
